## Curso: Ecuaciones Diferenciales Parciales
### Oscar Reula & Manuel Tiglio

## Movimiento de una Partícula Cargada en el Campo de un Dipolo Magnetico

### Numerically computing the orbits of electrons in a external dipolar magnetic field configuration.


The reference for this motion is: --Trapped Orbits in a Magnetic Dipole Field--, A. J. Dragt, Review of Geophysics, Vol. 3, Number 2, (1965), and --Global aspects of charged particle motion in axially symmetric multipole magnetic fields--, John V. Shebalin.The charge particle Lagrangian for a field with azimuthal symmetry is given by:
\begin{equation}
\mathcal{L} = -mc^2\sqrt{1-v^2/c^2} + \frac{e}{c}A(r,\theta)r\sin{\theta} v^{\phi},
\end{equation}
where $A$ is the $z$ component of the vector potential.

Recall that in these coordinates we have, $v^2 = \dot{r}^2 + r^2 \dot{\theta}^2 + r^2\sin^2(\theta) \dot{\phi}^2$. It is convenient to redefine $A \to A/c$ $ m = 1 $, $ e \to e/m$, to get rid of irelevant parameters.

Since the Lagrangian does not depend on time the energy is conserved. For this case, without and electrostatic potential, it is given by:

\begin{equation}
H = \frac{c^2}{\sqrt{1-v^2/c^2}} = \gamma c^2.
\end{equation}
Thus, the speed is conserved along the motion. On the other hand, the azimuthal symmetry also implies the particle angular momentum is also conserved,

\begin{equation}
P := \frac{\partial \mathcal{L}}{\partial \dot{\phi}} = \frac{\dot{\phi} r^2 \sin^2{\theta}}{\sqrt{1-v^2/c^2} }+ e A(r,\theta) r \sin{\theta}  = \gamma \dot{\phi} r^2 \sin^2{\theta} + e A(r,\theta) r \sin{\theta},
\end{equation}

In spherical coordinates the complete system of equations becomes:


\begin{eqnarray}
\frac{\partial \phi}{dt} & = & \frac{h}{\gamma r\sin \theta}, \quad h = \left[\frac{P}{r\sin{\theta}} - e A\right]  \\
\frac{\partial \dot{r}}{dt} & = & e\frac{1}{\gamma}\frac{\partial(rA)}{\partial r} \sin{\theta} \dot{\phi} + \frac{v^2 - \dot{r}^2}{r}  \\
\frac{\partial \dot{\theta}}{dt} & = & \left[\frac{e}{\gamma} \frac{1}{r}\frac{\partial(\sin{\theta}A)}{\partial\theta} + \sin{\theta}\cos{\theta} \dot{\phi}\right]\dot{\phi} - 2\frac{\dot{r}}{r}\dot{\theta}
\end{eqnarray}



Since the energy conservation implies the conservation of v2, using the expression for the angular momentum we obtain,
\begin{equation}
v^2 = \dot{r}^2 + r^2 \dot{\theta}^2 + r^2 \sin^2{\theta} \dot{\phi}^2 \dot{r}^2 + r^2 \dot{\theta}^2 = v^2 - \frac{h^2}{\gamma^2}.
\end{equation}



Therefore the movement is only possible when $h^2 = \left[\frac{P}{r\sin{\theta}} - e A\right]^2 < v^2\gamma^2$. For a generic multipole we have, 
\begin{equation}
A(r,\theta) = \frac{a_n}{r^{n+1}}P^1_n(\cos{\theta}).
\end{equation}

### El Caso Dipolar

Para el caso, $ P^1_1(\cos \theta) = -\sin(\theta) $

Definimos $e a_1 = a_0 $

\begin{eqnarray}
\frac{\partial \phi}{dt} & = & \frac{h}{\gamma r\sin{\theta}} \quad h = \left[\frac{P}{r\sin{\theta}} + \frac{a_0}{r^2}\sin{\theta}\right] \\
\frac{\partial \dot{r}}{dt} & = &  \frac{1}{\gamma}\frac{a_0}{r^2} \sin^2{\theta} \dot{\phi} + \frac{v^2 - \dot{r}^2}{r} \\
\frac{\partial \dot{\theta}}{dt} & = & \left[- \frac{2a_0}{\gamma r^3} +  \dot{\phi}\right]\dot{\phi}\sin{\theta}\cos{\theta}  - 2\frac{\dot{r}}{r}\dot{\theta} 
\end{eqnarray}

Donde

\begin{equation}
v^2 = \frac{\dot{r}^2 + r^2 \dot{\theta}^2 + h^2}{1+h^2},
\end{equation}

es constante a lo largo del movimiento. Por tanto, únicamente necesitamos resolver para
$ U = (r, \dot{r}, \theta, \dot{\theta}) $, pero también resolvemos a lo largo de
$ \phi $ tal que podemos fácilmente graficar las soluciones. Por lo tanto, nuestra solución
vectorial es $ U = (r, \dot{r}, \theta, \dot{\theta}) $.

In [7]:
# Computa numericamente las orbitas de un electron en un campo magnetico externo de forma
# dipolar
#
import matplotlib as plt
import numpy as np
import matplotlib
#matplotlib.use('TKAgg')
import matplotlib.pyplot as plt
from scipy.integrate import ode
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D

 Here we put some of the parameters that enter the equations. For slow particles (simpler to evolve, but with all the complexity) $ a = 0.1 $, pert $= 0.01$, T=6000−20000, Tol=10−8. For faster particles (harder to evolve) a=1.0, pert=0.1, T=200−400, Tol=10−10. The fastest is a=10.0, pert=0.8, T=10−20, Tol=10−12. This is very hard for the integrator we are using. I recomment to play with the slowest one, it has a nicer structure.

In [8]:
a = 10.0  # field amplitude (reescaled with e, m and c)
#a = 0.1
p = -1. * a # angular momentum (reescaled with m and a). 
# With this value there is a part of h^2 < 1 which is disconected from the outside
#pert = 0.01 # this gives bounded orbits. This value enters in theta_dot, so it is de departure from plane orbits
#pert = 0.1
pert = 0.8 # for faster particles
# Integration time
#T_f = 6000
T_f = 200
#T_f=2
# maximum step error on the integration
reltol=1e-8
#abstol=1e-8; # for slower
#reltol=1e-10
#abstol=1e-10; # for slower
#reltol=1e-12
abstol=1e-12; # for faster

 Next we give the initial conditions. We start from the equatorial plane, at r=1. With no radial velocity. The off-plane velocity is small, and is given by the pert parameter. The ϕ velocity is obtained from the value of the given angular momentum. Thus, we know the total velocity after wards, and we need to check it is smaller that the speed of light!

In [9]:
# Primero las condiciones iniciales U=(rho,rho_t,theta, theta_t,phi)
N=5
T_0 = 0.
rho0 = 1.
#rho_t0 = -0.02
rho_t0 = -0.0
#pert = 0.000000000000000
pert = 0.8
theta0 = np.pi/2.0
theta_t0 = pert
phi0 = 0

h0 = (p/rho0/np.sin(theta0) + a*np.sin(theta0)/rho0/rho0)
h20 = h0*h0
v2 = (rho_t0*rho_t0 + rho0*rho0*(theta_t0*theta_t0) + h20) / (1.+h20)

if (v2 > 0.99):
    print('velocidad demasiado alta! v2=%f', v2)
    exit()
gamma = np.sqrt(1.-v2)                        

u0 = [rho0, rho_t0, theta0, theta_t0, phi0]

 We now plot the regions $ h^2 − 1 = $ constant = $\left[\frac{P}{r\sin \theta} + \frac{a_0}{r^2}\sin \theta \right]^2 − 1$ to see where are the bounded, and possible, propagation regions. Namely those regions for wich $h^2 - 1 < 0$ (Only in the Julia version)

In [10]:
# Luego definimos la funcion del integrador, o sea la ecuacion

def f(x, u):
    sint = np.sin(u[2])
    cost = np.cos(u[2])
    phi_t = (p/u[0]/sint + a/u[0]/u[0]*sint)/u[0]/sint/gamma
    
    return [u[1], a/u[0]/u[0]*sint*sint*phi_t/gamma + (v2 - u[1]*u[1])/u[0],u[3],
            (-2.*a/u[0]/u[0]/u[0]/gamma + phi_t)*phi_t*sint*cost - 2.*u[1]*u[3]/u[0],phi_t]